In [1]:

import sys
import os
sys.path.append('..')

from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Load environment variables
load_dotenv()

# Import our custom functions
from scripts.document_processor import load_pdf, chunk_text, load_pdfs_from_directory
# from scripts.embedding_manager import create_embeddings, setup_vector_store
from scripts.embedding_manager_med import create_embeddings, setup_vector_store
from scripts.mcq_processor import MCQProcessor, MCQInput
from scripts.mcq_generator import generate_mcq_dataset

import json
import os
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

/Users/binitkc/RAG_Proj/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:

# Load all PDFs from data directory
data_dir = '../data'
documents = load_pdfs_from_directory(data_dir)
print(f"Loaded total {len(documents)} pages from all PDFs")


Loaded 11 pages from sacks-et-al-2006-soy-protein-isoflavones-and-cardiovascular-health.pdf
Loaded 8 pages from 868-Article Text-1633-2-10-20170528.pdf
Loaded 14 pages from Convulsive Status Epliepsy.pdf
Loaded 23 pages from metlay-et-al-2019-diagnosis-and-treatment-of-adults-with-community-acquired-pneumonia-an-official-clinical-practice.pdf
Loaded 9 pages from silberstein-et-al-2012-evidence-based-guideline-update-pharmacologic-treatment-for-episodic-migraine-prevention-in.pdf
Loaded 80 pages from Primary Open-Angle Glaucoma PPP.pdf
Loaded 39 pages from Adult Sinusitis.pdf
Loaded 16 pages from Rheumatoid Arthritis.pdf
Loaded 17 pages from nonalcoholic-fatty-liver-disease-nafld-assessment-and-management-pdf-1837461227461.pdf


Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 48 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 66 0 (offset 0)


Loaded 4 pages from diabetes_guidelines.pdf
Loaded 8 pages from AIC_Tapering_and_Testing_Guide.pdf
Loaded 25 pages from Acne.pdf
Loaded 66 pages from World Federation of Societies of Biological Psychiatry  WFSBP  Guidelines for the Pharmacological Treatment of Anxiety  Obsessive-Compulsive and Post-.pdf
Loaded 2 pages from CommonCold_fact_sheet_508.pdf
Loaded 35 pages from LiverFailure-English-report.pdf
Loaded 199 pages from KDIGO-2024-CKD-Guideline.pdf
Loaded 28 pages from hematology.pdf
Loaded 37 pages from ESPEN_guideline_on_clinical_nutrition_in_liver_disease.pdf
Loaded 64 pages from gonorrhoeae.pdf
Loaded 11 pages from AcuteAltitudeSickness.pdf
Loaded total 696 pages from all PDFs


In [3]:

chunks = chunk_text(documents)
# print(f"Created {len(chunks)} chunks")
print(f"Processed {len(documents)} pages into {len(chunks)} chunks")

# Display one chunk as example
print("\nExample chunk:")
print(chunks[12].page_content[:200])

Processed 696 pages into 4458 chunks

Example chunk:
could have had a blood cholesterol–lowering action. Results
of other early studies of soy protein in hypercholesterolemic
subjects showed either cholesterol reduction12 or no
effect.13,14
A meta-analy


In [4]:
# Initialize embeddings
embeddings = create_embeddings()

# Setup vector store
vectordb = setup_vector_store(chunks, embeddings)
print("Vector store created successfully")



/Users/binitkc/RAG_Proj/notebooks/../scripts/embedding_manager.py:19: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  return OpenAIEmbeddings(model="aaditya/OpenBioLLM-Llama3-70B")


Exception: Error setting up vector store: Error code: 400 - {'error': {'message': 'invalid model ID', 'type': 'invalid_request_error', 'param': None, 'code': None}}